In [2]:
import scipy
import time
import pandas as pd
import numpy as np

In [3]:
data_user = pd.read_csv('data_user.csv')

In [4]:
data_user

,Unit,Timestep,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,Timestamp
0,176,1,42.0003,0.8400,100.0,445.00,549.37,1345.94,1128.55,3.91,...,2387.95,8082.22,9.3796,0.02,331,2212,100.00,10.76,6.3382,2023-01-30
1,176,2,0.0021,0.0000,100.0,518.67,642.25,1583.77,1402.43,14.62,...,2388.12,8138.43,8.4156,0.03,394,2388,100.00,39.06,23.3051,2023-01-31
2,176,3,25.0002,0.6212,60.0,462.54,537.03,1265.74,1043.04,7.05,...,2028.26,7874.99,10.8821,0.02,306,1915,84.93,14.41,8.7017,2023-02-01
3,176,4,41.9994,0.8400,100.0,445.00,549.93,1350.62,1131.21,3.91,...,2387.97,8090.01,9.3628,0.02,330,2212,100.00,10.49,6.2807,2023-02-02
4,176,5,34.9998,0.8400,100.0,449.44,555.71,1371.53,1130.00,5.48,...,2388.04,8071.90,9.2974,0.02,334,2223,100.00,14.82,8.9769,2023-02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9415,223,214,0.0018,0.0017,100.0,518.67,643.18,1600.42,1418.62,14.62,...,2388.15,8210.17,8.4818,0.03,396,2388,100.00,38.50,23.0476,2023-08-31
9416,223,215,10.0064,0.2510,100.0,489.05,605.60,1513.96,1331.70,10.52,...,2388.39,8200.92,8.7295,0.03,374,2319,100.00,28.15,16.9650,2023-09-01
9417,223,216,35.0013,0.8401,100.0,449.44,556.84,1377.24,1144.45,5.48,...,2389.56,8145.75,9.3906,0.02,338,2223,100.00,14.86,8.8668,2023-09-02
9418,223,217,0.0021,0.0000,100.0,518.67,643.53,1602.01,1432.23,14.62,...,2388.17,8211.58,8.5271,0.03,395,2388,100.00,38.51,23.1052,2023-09-03


In [7]:
machine = data_user[data_user.Unit == 176].copy().reset_index(drop=True)

In [9]:
series = data_user.iloc[0:1]

In [12]:
df_day = data_user[data_user.Timestamp == "2023-01-30"]

In [13]:
df_day.head()

,Unit,Timestep,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,Timestamp
0,176,1,42.0003,0.84,100.0,445.00,549.37,1345.94,1128.55,3.91,...,2387.95,8082.22,9.3796,0.02,331,2212,100.0,10.76,6.3382,2023-01-30
164,177,1,10.0011,0.25,100.0,489.05,605.03,1501.48,1308.10,10.52,...,2388.18,8122.33,8.6732,0.03,369,2319,100.0,28.68,17.1517,2023-01-30
345,178,1,42.0020,0.84,100.0,445.00,549.35,1343.24,1120.84,3.91,...,2387.85,8077.82,9.3637,0.02,330,2212,100.0,10.41,6.4128,2023-01-30
493,179,1,35.0061,0.84,100.0,449.44,555.43,1366.11,1131.00,5.48,...,2387.76,8056.99,9.3388,0.02,334,2223,100.0,14.75,8.8769,2023-01-30
674,180,1,0.0027,0.00,100.0,518.67,641.91,1583.79,1395.41,14.62,...,2388.00,8138.23,8.4227,0.03,391,2388,100.0,39.08,23.5000,2023-01-30


In [38]:
from joblib import dump, load
pd.options.mode.chained_assignment = None 
import warnings
warnings.filterwarnings("ignore")
conditionVariables = ['op_setting_1', 'op_setting_2', 'op_setting_3']
dataVariables = ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
        'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10',
        'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15',
        'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
        'sensor_21']

In [27]:
model_kmeans = load('model_kmeans.joblib')

In [28]:
opCondition = df_day[conditionVariables] # get operating conditions

In [29]:
y_cluster = model_kmeans.predict(opCondition)

In [30]:
df_day['cluster'] = y_cluster ## Create cruster

In [31]:
df_day

,Unit,Timestep,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,Timestamp,cluster
0,176,1,42.0003,0.8400,100.0,445.00,549.37,1345.94,1128.55,3.91,...,8082.22,9.3796,0.02,331,2212,100.00,10.76,6.3382,2023-01-30,0
164,177,1,10.0011,0.2500,100.0,489.05,605.03,1501.48,1308.10,10.52,...,8122.33,8.6732,0.03,369,2319,100.00,28.68,17.1517,2023-01-30,4
345,178,1,42.0020,0.8400,100.0,445.00,549.35,1343.24,1120.84,3.91,...,8077.82,9.3637,0.02,330,2212,100.00,10.41,6.4128,2023-01-30,0
493,179,1,35.0061,0.8400,100.0,449.44,555.43,1366.11,1131.00,5.48,...,8056.99,9.3388,0.02,334,2223,100.00,14.75,8.8769,2023-01-30,5
674,180,1,0.0027,0.0000,100.0,518.67,641.91,1583.79,1395.41,14.62,...,8138.23,8.4227,0.03,391,2388,100.00,39.08,23.5000,2023-01-30,3
971,181,1,9.9989,0.2510,100.0,489.05,604.12,1494.92,1300.03,10.52,...,8145.75,8.6279,0.03,368,2319,100.00,28.47,17.2067,2023-01-30,4
1226,182,1,25.0050,0.6203,60.0,462.54,536.44,1263.10,1046.39,7.05,...,7881.93,10.8683,0.02,307,1915,84.93,14.26,8.4946,2023-01-30,2
1426,183,1,0.0024,0.0016,100.0,518.67,642.22,1591.27,1407.60,14.62,...,8119.73,8.4071,0.03,392,2388,100.00,38.95,23.3281,2023-01-30,3
1633,184,1,35.0051,0.8400,100.0,449.44,555.75,1364.91,1129.18,5.48,...,8061.15,9.2719,0.02,332,2223,100.00,14.96,8.8805,2023-01-30,5
1804,185,1,34.9992,0.8411,100.0,449.44,555.52,1357.78,1129.52,5.48,...,8068.20,9.3290,0.02,333,2223,100.00,14.80,9.0030,2023-01-30,5


In [39]:
def load_Normalization(data):
    list_df = []
    Clusters = data.cluster.unique()
    for cluster in Clusters:
        each_cluster = data[data.cluster == cluster].copy()
        sc_model = f'sc_model_{cluster}.joblib'
        sc = load(sc_model)
        each_cluster.loc[:, dataVariables] = sc.transform(each_cluster.loc[:,dataVariables])
        list_df.append(each_cluster)
    res_df = pd.concat(list_df).reset_index(drop=True)
    res_df.sort_values(by = ['Unit','Timestep'], inplace=True)
    return res_df

In [60]:
get_unit = data_user.groupby(by='Unit').max()['Timestep']

In [69]:
get_unit[get_unit > 200].index.values

array([180, 181, 183, 189, 190, 191, 196, 198, 199, 200, 203, 204, 206,
       209, 211, 215, 216, 218, 220, 223])

In [72]:
data_large = data_user[data_user.Unit.isin(get_unit[get_unit > 200].index.values)].copy().reset_index(drop=True)

In [73]:
data_large

,Unit,Timestep,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,Timestamp
0,180,1,0.0027,0.0000,100.0,518.67,641.91,1583.79,1395.41,14.62,...,2388.00,8138.23,8.4227,0.03,391,2388,100.0,39.08,23.5000,2023-01-30
1,180,2,10.0014,0.2505,100.0,489.05,604.43,1496.64,1295.81,10.52,...,2388.06,8133.21,8.5818,0.03,367,2319,100.0,28.71,17.2949,2023-01-31
2,180,3,42.0078,0.8400,100.0,445.00,549.21,1345.88,1121.68,3.91,...,2388.05,8084.77,9.3142,0.02,329,2212,100.0,10.64,6.2783,2023-02-01
3,180,4,42.0050,0.8400,100.0,445.00,548.91,1355.87,1115.86,3.91,...,2388.03,8083.02,9.3130,0.02,329,2212,100.0,10.44,6.3054,2023-02-02
4,180,5,20.0066,0.7011,100.0,491.19,606.55,1476.54,1236.67,9.35,...,2388.03,8065.78,9.1646,0.02,365,2324,100.0,24.38,14.7607,2023-02-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,223,214,0.0018,0.0017,100.0,518.67,643.18,1600.42,1418.62,14.62,...,2388.15,8210.17,8.4818,0.03,396,2388,100.0,38.50,23.0476,2023-08-31
4616,223,215,10.0064,0.2510,100.0,489.05,605.60,1513.96,1331.70,10.52,...,2388.39,8200.92,8.7295,0.03,374,2319,100.0,28.15,16.9650,2023-09-01
4617,223,216,35.0013,0.8401,100.0,449.44,556.84,1377.24,1144.45,5.48,...,2389.56,8145.75,9.3906,0.02,338,2223,100.0,14.86,8.8668,2023-09-02
4618,223,217,0.0021,0.0000,100.0,518.67,643.53,1602.01,1432.23,14.62,...,2388.17,8211.58,8.5271,0.03,395,2388,100.0,38.51,23.1052,2023-09-03
